In [2]:
import warnings

import pandas as pd
import numpy as np

from jre_utils.datapath import (
    model_built_data_paths,
)

from sklearn.metrics import r2_score

from jre_utils.process import get_most_active_municipalities
from jre_utils.backtest import Portfolio, Timeline


warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

In [3]:
# Other strategies
# E.g.
# Sort by cumulative taxable income
# Sort by momentum
# Sort by mean reversion

In [4]:
asset_type = "all"

metrics = {
    "median": "unit_price_median",
}

granularity_columns = ["area", "area_code", "asset_type"]
group_by_columns = granularity_columns + ["year"]
display_columns = ["unit_price", "total_traded_area", "count"]

metric_key = f"median"
metric = metrics[metric_key]

metric_pct_chg = metric + "_pct_chg"
normalized_metric_pct_chg = metric_pct_chg + "_normalized_yearly"

In [5]:
dataset_key = "transactions"
years_ahead = 2
dataset_name = f"sequence_{dataset_key}_{asset_type}_{metric_key}_{years_ahead}"


In [6]:
n = 500 # 500
investment_asset_type = "building"
model_built_data_path = model_built_data_paths[f"sequence_{dataset_key}_{investment_asset_type}_{metric_key}_{years_ahead}"]
core_df = pd.read_csv(model_built_data_path)

liquid_areas = get_most_active_municipalities(core_df, n)["area_code"].unique()

In [7]:
engineering_columns = [
    "net_migration_ratio",
    "taxable_income_growth",
    "new_dwellings_ratio",
    "total_tax_growth",
    "count_growth",
    "yearly_price_growth",
]

engineered_columns = []

for col in engineering_columns:
    core_df["multiplier"] = core_df[col] + 1
    core_df[f"{col}_ma3"] = core_df.groupby(granularity_columns)[col].transform(
        lambda x: x.rolling(3, 1).mean()
    )
    core_df[f"{col}_cumu3"] = core_df.groupby(granularity_columns)["multiplier"].transform(
        lambda x: x.rolling(3, 1).apply(np.prod, raw=True)
    )

    engineered_columns.append(f"{col}_ma3")
    engineered_columns.append(f"{col}_cumu3")
    print(f"{col}_ma3")
    print(f"{col}_cumu3")

core_df[engineered_columns] = core_df[engineered_columns].fillna(0)

net_migration_ratio_ma3
net_migration_ratio_cumu3
taxable_income_growth_ma3
taxable_income_growth_cumu3
new_dwellings_ratio_ma3
new_dwellings_ratio_cumu3
total_tax_growth_ma3
total_tax_growth_cumu3
count_growth_ma3
count_growth_cumu3
yearly_price_growth_ma3
yearly_price_growth_cumu3


In [189]:
# combining_factors
# core_df["combined_factor"] = core_df["taxable_income_growth_cumu3"] + core_df["net_migration_ratio_cumu3"] - core_df["unit_price_median_pct_chg_cumu3"]
# core_df["combined_factor"] = core_df["taxable_income_growth_cumu3"] + core_df["net_migration_ratio_cumu3"] - core_df["yearly_price_growth_cumu3"]
# core_df["combined_factor"] = -core_df["yearly_price_growth_cumu3"]

In [190]:
current_year_df = core_df[core_df["year"] == 2012]
current_year_df.head()

,area_code,area,year,asset_type,unit_price_median_pct_chg,count_growth,yearly_price_growth,total_tax_growth,taxable_income_growth,taxable_income_per_taxpayer_growth,net_migration_ratio,new_dwellings_ratio,taxpayer_count_growth,unit_price_median,count,total_traded_area,in_migrations,out_migrations,population,taxpayer_count,taxable_income,taxable_income_per_taxpayer,total_tax,new_dwellings,existing_dwellings,unit_price_median_log_normalized_yearly,count_log_normalized_yearly,total_traded_area_log_normalized_yearly,in_migrations_log_normalized_yearly,out_migrations_log_normalized_yearly,population_log_normalized_yearly,taxpayer_count_log_normalized_yearly,taxable_income_log_normalized_yearly,taxable_income_per_taxpayer_log_normalized_yearly,total_tax_log_normalized_yearly,new_dwellings_log_normalized_yearly,existing_dwellings_log_normalized_yearly,unit_price_median_pct_chg_normalized_yearly,count_growth_normalized_yearly,yearly_price_growth_normalized_yearly,total_tax_growth_normalized_yearly,taxable_income_growth_normalized_yearly,taxable_income_per_taxpayer_growth_normalized_yearly,net_migration_ratio_normalized_yearly,new_dwellings_ratio_normalized_yearly,taxpayer_count_growth_normalized_yearly,migrations_is_available,taxable_income_is_available,dwellings_is_available,total_tax_is_available,metric_pct_chg_is_available,multiplier,net_migration_ratio_ma3,net_migration_ratio_cumu3,taxable_income_growth_ma3,taxable_income_growth_cumu3,new_dwellings_ratio_ma3,new_dwellings_ratio_cumu3,total_tax_growth_ma3,total_tax_growth_cumu3,count_growth_ma3,count_growth_cumu3,unit_price_median_pct_chg_ma3,unit_price_median_pct_chg_cumu3,net_migration_ratio_normalized_yearly_ma3,net_migration_ratio_normalized_yearly_cumu3,taxable_income_growth_normalized_yearly_ma3,taxable_income_growth_normalized_yearly_cumu3,new_dwellings_ratio_normalized_yearly_ma3,new_dwellings_ratio_normalized_yearly_cumu3,total_tax_growth_normalized_yearly_ma3,total_tax_growth_normalized_yearly_cumu3,count_growth_normalized_yearly_ma3,count_growth_normalized_yearly_cumu3,taxable_income_growth_cumu3+net_migration_ratio_cumu3,taxable_income_growth_cumu3*net_migration_ratio_cumu3,combined_factor,yearly_price_growth_ma3,yearly_price_growth_cumu3
4,23441,Aichi-ken Agui-cho,2012,building,0.026316,0.100000,0.026316,0.010660,0.032618,0.007802,0.012503,NaN,0.024624,185714.285714,44.0,9470.0,1142.0,807.0,26793.0,11984.0,39195622.0,3270.662717,3842470.0,NaN,NaN,1.247363,0.108099,-0.155149,0.140714,-0.150688,-0.011302,0.139754,0.244937,1.229065,0.184986,NaN,NaN,-0.042394,-0.107277,-0.072135,0.354172,0.737669,0.427423,2.077046,NaN,0.555778,1,1,0,1,1,1.026316,0.018263,1.055770,-0.010867,0.963065,0.000000,0.000000,0.006330,1.017601,0.163134,1.517241,0.129556,1.413584,2.384063,37.097843,0.305070,1.098465,0.000000,0.000000,0.188537,0.670472,0.199149,1.529827,0.605250,1.016775,-1.075188,0.024645,1.075188
19,23232,Aichi-ken Aisai-shi,2012,building,0.088000,-0.024390,0.105000,-0.004037,0.001476,-0.002873,0.000712,0.014550,0.004362,102000.000000,40.0,9315.0,1607.0,1563.0,61807.0,28324.0,84092410.0,2968.945417,7147834.0,323.0,22199.0,0.643184,0.029793,-0.170664,0.350608,0.279592,0.531729,0.694702,0.711334,0.582690,0.572548,-0.359231,-0.615203,0.079533,-0.373428,0.090108,-0.036486,0.090000,-0.011867,0.527612,0.032865,0.099804,1,1,1,1,1,1.105000,-0.000316,0.999047,-0.035284,0.895134,0.013749,1.041817,-0.016835,0.949726,-0.062552,0.816327,-0.165030,0.545628,0.374475,2.468224,-0.328928,0.145429,-0.007255,0.963319,-0.364659,-0.043693,-0.360002,0.261527,1.348553,0.894281,-0.805263,-0.056751,0.805263
34,23237,Aichi-ken Ama-shi,2012,building,-0.214488,0.268657,-0.117647,0.013124,-0.001269,-0.008801,-0.000945,0.016311,0.007599,117647.058824,85.0,18075.0,2978.0,3059.0,85730.0,38188.0,114177490.0,2989.878758,10143499.0,586.0,35927.0,0.787077,0.652920,0.452571,0.729767,0.716862,0.744290,0.887496,0.898196,0.629612,0.791103,0.125113,-0.095642,-0.518374,0.253589,-0.368980,0.419677,0.032921,-0.255789,0.309912,

In [191]:
investment_start_year = 2012
investment_end_year = 2020
investment_eval_end_year = 2022

rebalancing_years = range(investment_start_year, investment_end_year + 1, 2)

asset_count = 20
shorting_enabled = False
cash, assets, liabilities = 1000, {}, {}

In [192]:
initial_portfolio = Portfolio(assets, liabilities, cash)
timeline = Timeline(investment_start_year, initial_portfolio)

factor_column = "combined_factor"

for year in range(investment_start_year, investment_eval_end_year + 1):
    current_year_df = core_df[core_df["year"] == year]
    # yearly_correlation_score = r2_score(current_year_df[normalized_metric_pct_chg], current_year_df["predicted_normalized_return"])
    current_year_df = current_year_df[current_year_df["asset_type"] == investment_asset_type]
    current_year_df = current_year_df.set_index('area_code')

    timeline.remark(year, current_year_df)

    print(f"{year} NAV: {timeline.get_current_portfolio().nav()}") # Correlation Score: {yearly_correlation_score}

    if year in rebalancing_years:
        factor_df = current_year_df[current_year_df.index.isin(liquid_areas)]
        top_areas = factor_df.nlargest(asset_count, factor_column).index
        bottom_areas = factor_df.nsmallest(asset_count, factor_column).index
        timeline.rebalance(top_areas, bottom_areas if shorting_enabled else [])


2012 NAV: 1000
2013 NAV: 1158.4371351732977
2014 NAV: 1208.2083031948694
2015 NAV: 1449.750114642069
2016 NAV: 1425.9008706181457
2017 NAV: 1799.5795151288191
2018 NAV: 1819.4608968505588
2019 NAV: 2480.3916820060576
2020 NAV: 2746.199053024685
2021 NAV: 3206.7807159682284
2022 NAV: 3389.0733900753626


In [193]:
rebalancing_ratios = timeline.calculate_asset_rebalancing_ratio()

2014 Rebalancing ratio: 0.90
2016 Rebalancing ratio: 0.90
2018 Rebalancing ratio: 0.95
2020 Rebalancing ratio: 0.90


In [194]:
print(f"Annualized Return: ", timeline.calculate_annualized_return())
print(f"Sharpe Ratio:", timeline.calculate_sharpe_ratio())

Annualized Return:  0.11734986128378955
Sharpe Ratio: 1.1971036297649191
